In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu, sobel
from skimage.morphology import remove_small_objects, disk, binary_closing
from scipy.ndimage import zoom, binary_dilation, binary_erosion
from skimage.measure import label, regionprops
from skimage import io, exposure, color
from skimage import measure, morphology
from skimage import exposure
from czifile import imread
from cellpose import models, plot 
import cv2
import re
model = models.Cellpose(gpu=False, model_type='cyto3')


c:\Users\sj1205\AppData\Local\Programs\Python\Python312\Lib\site-packages\cellpose\resnet_torch.py:275: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load

In [8]:
def display_image(image, path, type):
    """Display the image."""
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{path} {type}")
    plt.show()

def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def extract_channels(image: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Extract green and red channels from the squeezed image (shape: [Z, C, H, W]).""" 
    return image[0], image[1]

def calculate_surface_area(labeled_image: np.ndarray) -> float:
    """Calculate the total surface area for labeled regions."""
    props = regionprops(labeled_image)
    return sum(prop.area for prop in props)

def remove_small_objects_only(binary_image, min_size=100):
    labeled_image = measure.label(binary_image)

    # Remove objects smaller than min_size
    filtered_image = morphology.remove_small_objects(labeled_image, min_size=min_size)

    # Convert back to binary mask
    return filtered_image > 0

def remove_small_objects(labeled_image, min_size=10):

    # Remove objects smaller than min_size
    filtered_image = morphology.remove_small_objects(labeled_image, min_size=min_size)

    # Convert back to binary mask
    return filtered_image > 0

def high_circularity_mask(labeled_image: np.ndarray, threshold: float = 0.7) -> np.ndarray:
    """
    Return a binary mask with only the regions that have a circularity index higher than the threshold.
    
    Parameters:
        labeled_image (np.ndarray): Labeled image where each object has a unique label.
        threshold (float): Minimum circularity index to keep the object.
        
    Returns:
        np.ndarray: Binary mask with only high-circularity objects.
    """
    mask = np.zeros_like(labeled_image, dtype=np.uint8)
    props = regionprops(labeled_image)

    for prop in props:
        perimeter = prop.perimeter
        area = prop.area
        if perimeter > 0:
            circularity = (4 * np.pi * area) / (perimeter ** 2)
            if circularity > threshold:
                # Add the object to the mask
                mask[labeled_image == prop.label] = 1

    return mask


def remove_skinny_objects(binary_image: np.ndarray, aspect_ratio_thresh=3.0, eccentricity_thresh=0.95) -> np.ndarray:
    """
    Removes long, skinny objects from a binary mask based on aspect ratio and eccentricity.
    
    Parameters:
        binary_image (np.ndarray): Binary image with objects to filter.
        aspect_ratio_thresh (float): Max allowed aspect ratio (e.g. 3.0).
        eccentricity_thresh (float): Max allowed eccentricity (e.g. 0.95).
        
    Returns:
        np.ndarray: Binary mask with skinny/elongated objects removed.
    """
    labeled = label(binary_image)
    output_mask = np.zeros_like(binary_image, dtype=np.uint8)

    for region in regionprops(labeled):
        minr, minc, maxr, maxc = region.bbox
        height = maxr - minr
        width = maxc - minc
        aspect_ratio = max(width / height, height / width)

        if aspect_ratio <= aspect_ratio_thresh and region.eccentricity <= eccentricity_thresh:
            output_mask[labeled == region.label] = 1

    return output_mask

def analysis(red: np.ndarray, green:np.ndarray, path:str) -> pd.DataFrame:
    data = []
    df_cell_summary = pd.DataFrame()

    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 2.25) #1.75 works well
    mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria
    mitochondria_thresholded = remove_small_objects_only(mitochondria_thresholded, min_size=10)


    if (re.search(r'LLOMe', path)):
        constrast_adjusted_green_normalized = (green - green.min()) / (green.max() - green.min())
        threshold_value_green = np.mean(constrast_adjusted_green_normalized) + (np.std(constrast_adjusted_green_normalized) * 3) #1.75 works well
        green_thresholded = constrast_adjusted_green_normalized > threshold_value_green
        green_thresholded = remove_small_objects_only(green_thresholded, min_size=10)
    else: 
        constrast_adjusted_green_normalized = (green - green.min()) / (green.max() - green.min())
        threshold_value_green = np.mean(constrast_adjusted_green_normalized) + (np.std(constrast_adjusted_green_normalized) * .9) #1.75 works well
        green_thresholded = constrast_adjusted_green_normalized > threshold_value_green
        green_thresholded = remove_small_objects_only(green_thresholded, min_size=35)

    red_surface_area = calculate_surface_area(label(mitochondria_thresholded))
    green_surface_area = calculate_surface_area(label(green_thresholded))
    overlap_surface_area = calculate_surface_area(label(mitochondria_thresholded) & label(green_thresholded))

    display_image(mitochondria_thresholded, path, "Red Thresholded")
    display_image(green_thresholded, path, "Green Thresholded")
    display_image(mitochondria_thresholded & green_thresholded, path, "Overlap")

    data.append({
        'Image Path': path,
        'Red Surface Area': red_surface_area,
        'Green Surface Area': green_surface_area,
        'Overlap Surface Area': overlap_surface_area,
        'Overlap / Red Surface Area': overlap_surface_area / red_surface_area if red_surface_area > 0 else 0,
        'Overlap / Green Surface Area': overlap_surface_area / green_surface_area if green_surface_area > 0 else 0,
    })
    df_cell_summary = pd.DataFrame(data)
    return df_cell_summary


def dox_analysis(red: np.ndarray, green:np.ndarray, path:str) -> pd.DataFrame:
    data = []
    inclusion_details = []


    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 4.5) #1.75 works well
    mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria
    mitochondria_thresholded = remove_small_objects_only(mitochondria_thresholded, min_size=10)

    #display_image(mitochondria_thresholded, path, "Red Thresholded")

    masks, flows, styles, diams = model.eval(green, diameter=200, channels=[0, 0])

    labeled_cells = label(masks)
    filtered_cells = [cell for cell in regionprops(labeled_cells) if cell.area >= 100]

    for i, cell in enumerate(filtered_cells):
        mask = labeled_cells == cell.label
        applied_mask = gaussian(green) * mask
        threshold = 0.75
        inclusions = applied_mask > threshold
        inclusions_labeled = label(inclusions)
        if len(np.unique(inclusions_labeled)) > 2:
            inclusions_filtered = remove_small_objects(inclusions_labeled, min_size=10)
        else:
            inclusions_filtered = inclusions_labeled

        inclusions_circularity = None
        inclusions_removed_skinny = None

        if re.search(r'noOA', path):
            inclusions_filtered = remove_skinny_objects(inclusions_filtered, aspect_ratio_thresh=2, eccentricity_thresh=0.95)
            inclusions_filtered = remove_small_objects(label(inclusions_filtered), min_size=20)
            inclusions_removed_skinny = True
        elif re.search(r'OA', path):
            inclusions_filtered = high_circularity_mask(label(inclusions_filtered), threshold=0.4)
            inclusions_filtered = remove_small_objects(label(inclusions_filtered), min_size=20)
            inclusions_circularity = True

        #if (inclusions_circularity is not None) or (inclusions_removed_skinny is not None):
        #    # display the filtered inclusions
        #    display_image(inclusions_filtered, path, "Filtered Inclusions cell " + str(i + 1))
        
        red_within_cell = mitochondria_thresholded * mask

        red_labeled = label(red_within_cell)
        green_labeled = label(inclusions_filtered)

        red_binary = red_labeled > 0
        green_binary = green_labeled > 0
        overlap_binary = red_binary & green_binary

        red_surface_area = calculate_surface_area(red_labeled)
        green_surface_area = calculate_surface_area(green_labeled)
        overlap_surface_area = calculate_surface_area(label(overlap_binary))  

        number_of_inclusions = len(regionprops(green_labeled))
        overlapping_inclusions = 0

        green_region_props = regionprops(green_labeled)
        for region in green_region_props:
            inclusion_mask = green_labeled == region.label
            if np.any(red_binary & inclusion_mask):
                overlapping_inclusions += 1

        for j, region in enumerate(green_region_props):
            inclusion_mask = green_labeled == region.label
            overlaps_with_red = np.any(red_binary & inclusion_mask)

            inclusion_details.append({
                'Image Path': path,
                'Cell ID': i + 1,
                'Inclusion ID': j + 1,
                'Inclusion Size': region.area,
                'Overlaps with Red': overlaps_with_red
            })
        

        data.append({
            'Image Path': path,
            'Cell ID': i + 1,
            'Number of Inclusions': number_of_inclusions,
            'Number of Overlapping Inclusions': overlapping_inclusions,
            'Red Surface Area': red_surface_area,
            'Green Surface Area': green_surface_area,
            'Overlap Surface Area': overlap_surface_area,
            'Overlap / Red Surface Area': overlap_surface_area / red_surface_area if red_surface_area > 0 else 0,
            'Overlap / Green Surface Area': overlap_surface_area / green_surface_area if green_surface_area > 0 else 0,
        })

    df_cell_summary = pd.DataFrame(data)
    df_inclusion_details = pd.DataFrame(inclusion_details)
    return df_cell_summary, df_inclusion_details
    





In [9]:
def main(image_folder):
    images_to_analyze = extract_image_paths(image_folder)
    output_dir = os.getcwd()

    df_summary = []
    df_inclusion_summary_total = []
    for path in images_to_analyze:
        image = read_image(path)
        image_squeezed = np.squeeze(image) 

        red, green = extract_channels(image_squeezed)
        if re.search(r'^(?!.*dox)', path):
            #df_cell_summary = analysis(red, green, path)
            #df_summary.append(df_cell_summary)
            continue
        else: 
            df_cell_summary, df_inclusion_summary = dox_analysis(red, green, path)
            df_summary.append(df_cell_summary)
            df_inclusion_summary_total.append(df_inclusion_summary)

    df_summary_combined_df = pd.concat(df_summary, ignore_index=True)
    output_df_summary_path = os.path.join(output_dir, '41125_galectin_experiment.xlsx')
    df_summary_combined_df.to_excel(output_df_summary_path, index=False)

    df_inclusion_summary_combined_df = pd.concat(df_inclusion_summary_total, ignore_index=True)
    output_df_inclusion_summary_path = os.path.join(output_dir, '41125_galectin_experiment_inclusions.xlsx')
    df_inclusion_summary_combined_df.to_excel(output_df_inclusion_summary_path, index=False)



if __name__ == "__main__":
    image_folder = '41125_galectin'
    main(image_folder)


C:\Users\sj1205\AppData\Local\Temp\ipykernel_35056\3373510019.py:37: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  filtered_image = morphology.remove_small_objects(labeled_image, min_size=min_size)
